In [ ]:


"""
まず、このままだと動かない。データ抽出に関わる部分抜き出しただけ。

基本的に、JSでデータを使用するためjson形式で格納している。
pythonのみで動かす場合は、json形式に直す必要性はない。変更は必須。

params_json部分に直接文字列を入れればなんとかなるはず...。

"""


"""
lblファイルデータの抽出関数。
必要ないと思う...。
"""
def base_json(params):

    # json形式のデータ
    params_json = params["properties"]
    geometry = params["geometry"]

    # 新しいjson形式fieldを定義 配列の宣言みたいなやつ
    field = cl.OrderedDict()

    field["path"] = params_json["path"]["data"]　#data -> cub path
    field["obs_ID"] = params_json["id"]

    #cubファイルオープン！！
    cube_data = gdal.Open(params_json["path"]["data"]["main"]["cub"], gdal.GA_ReadOnly) #cub file sonomama

    # 新しいjson形式dataを定義
    data = cl.OrderedDict()

    # lblファイルの中身を取り出して、json形式で格納！！
    lbl_data = pvl.load(params_json["path"]["data"]["main"]["lbl"]) #lbl name wo sonomama
    """
    上記は、crism trr のlblファイルの読み込み。
    以降、lbl_data["オブジェクト名"]["オブジェクト名"]...はlblファイルのオブジェクト名に相当する。
    """
    
    ######## CRISM #########
    if params_json["name"] == 'crism':
        field["obs_name"] = str(lbl_data["INSTRUMENT_ID"])
        data["PRODUCT_TYPE"] = str(lbl_data["PRODUCT_TYPE"])
        data["INSTRUMENT_HOST_NAME"] = str(lbl_data["INSTRUMENT_HOST_NAME"])
        data["SPACECRAFT_ID "] = str(lbl_data["SPACECRAFT_ID"])
        data["MRO:FRAME_RATE"] = " ".join(map(str,lbl_data["MRO:FRAME_RATE"]))
        data["MRO:EXPOSURE_PARAMETER"] = str(lbl_data["MRO:EXPOSURE_PARAMETER"])
        data["SOLAR_DISTANCE"] = " ".join(map(str,lbl_data["SOLAR_DISTANCE"]))

        filter_num = int(lbl_data['MRO:WAVELENGTH_FILTER']) + 1
        cdr = np.genfromtxt(params_json["path"]["data"]["derived"]["wavelength"], delimiter = "," ,dtype = np.float,usecols = (1))
        filter_rf = np.genfromtxt(params_json["path"]["data"]["derived"]["filter"], delimiter = ",",dtype = np.uint16,usecols = (filter_num))
        usedwv = []
        num_band = 1
        while num_band <filter_rf.shape[0]:
            if filter_rf[num_band] == 1:
                usedwv.append(cdr[num_band]/1000)
            num_band  =  num_band+1

        data["band_bin_center"] = ",".join(map(str, usedwv))
        data2 = cl.OrderedDict()
        data2["Image_size"] = [cube_data.RasterXSize,cube_data.RasterYSize]
        field["Mapping"] = data2
        field["Image_path"] = params_json["path"]["image"]["thumbnail"]
        field["Ratio_path_json"] = params_json["path"]["image"].get('ratio')
        field["geometry"] = geometry
    else:
        return "NOdata"
    
    data["StartTime"] = str(lbl_data["Caminfo"]["Geometry"]["StartTime"])
    data["EndTime"] = str(lbl_data["Caminfo"]["Geometry"]["EndTime"])
    data["CenterLatitude"] = str(lbl_data["Caminfo"]["Geometry"]["CenterLatitude"])
    data["CenterLongitude"] = str(lbl_data["Caminfo"]["Geometry"]["CenterLongitude"])
    data["CenterRadius"] = str(lbl_data["Caminfo"]["Geometry"]["CenterRadius"])
    data["RightAscension"] = str(lbl_data["Caminfo"]["Geometry"]["RightAscension"])
    data["Declination"] = str(lbl_data["Caminfo"]["Geometry"]["Declination"])
    data["UpperLeftLongitude"] = str(lbl_data["Caminfo"]["Geometry"]["UpperLeftLongitude"])
    data["UpperLeftLatitude"] = str(lbl_data["Caminfo"]["Geometry"]["UpperLeftLatitude"])
    data["LowerLeftLongitude"] = str(lbl_data["Caminfo"]["Geometry"]["LowerLeftLongitude"])
    data["LowerLeftLatitude"] = str(lbl_data["Caminfo"]["Geometry"]["LowerLeftLatitude"])
    data["LowerRightLongitude"] = str(lbl_data["Caminfo"]["Geometry"]["LowerRightLongitude"])
    data["LowerRightLatitude"] = str(lbl_data["Caminfo"]["Geometry"]["LowerRightLatitude"])
    data["UpperRightLongitude"] = str(lbl_data["Caminfo"]["Geometry"]["UpperRightLongitude"])
    data["UpperRightLatitude"] = str(lbl_data["Caminfo"]["Geometry"]["UpperRightLatitude"])

    data["PhaseAngle"] = str(lbl_data["Caminfo"]["Geometry"]["PhaseAngle"])
    data["EmissionAngle"] = str(lbl_data["Caminfo"]["Geometry"]["EmissionAngle"])
    data["IncidenceAngle"] = str(lbl_data["Caminfo"]["Geometry"]["IncidenceAngle"])
    data["NorthAzimuth"] = str(lbl_data["Caminfo"]["Geometry"]["NorthAzimuth"])
    data["OffNadir"] = str(lbl_data["Caminfo"]["Geometry"]["OffNadir"])
    data["SolarLongitude"] = str(lbl_data["Caminfo"]["Geometry"]["SolarLongitude"])
    data["LocalTime"] = str(lbl_data["Caminfo"]["Geometry"]["LocalTime"])
    data["TargetCenterDistance"] = str(lbl_data["Caminfo"]["Geometry"]["TargetCenterDistance"])
    data["SlantDistance"] = str(lbl_data["Caminfo"]["Geometry"]["SlantDistance"])
    data["SampleResolution"] = str(lbl_data["Caminfo"]["Geometry"]["SampleResolution"])
    data["LineResolution"] = str(lbl_data["Caminfo"]["Geometry"]["LineResolution"])
    data["PixelResolution"] = str(lbl_data["Caminfo"]["Geometry"]["PixelResolution"])
    data["MeanGroundResolution"] = str(lbl_data["Caminfo"]["Geometry"]["MeanGroundResolution"])
    data["SubSolarAzimuth"] = str(lbl_data["Caminfo"]["Geometry"]["SubSolarAzimuth"])
    data["SubSolarGroundAzimuth"] = str(lbl_data["Caminfo"]["Geometry"]["SubSolarGroundAzimuth"])
    data["SubSolarLatitude"] = str(lbl_data["Caminfo"]["Geometry"]["SubSolarLatitude"])
    data["SubSolarLongitude"] = str(lbl_data["Caminfo"]["Geometry"]["SubSolarLongitude"])
    data["SubSpacecraftAzimuth"] = str(lbl_data["Caminfo"]["Geometry"]["SubSpacecraftAzimuth"])
    data["SubSpacecraftGroundAzimuth"] = str(lbl_data["Caminfo"]["Geometry"]["SubSpacecraftGroundAzimuth"])
    data["SubSpacecraftLatitude"] = str(lbl_data["Caminfo"]["Geometry"]["SubSpacecraftLatitude"])
    data["SubSpacecraftLongitude"] = str(lbl_data["Caminfo"]["Geometry"]["SubSpacecraftLongitude"])
    data["ParallaxX"] = str(lbl_data["Caminfo"]["Geometry"]["ParallaxX"])
    data["ParallaxY"] = str(lbl_data["Caminfo"]["Geometry"]["ParallaxY"])
    data["ShadowX"] = str(lbl_data["Caminfo"]["Geometry"]["ShadowX"])
    data["ShadowY"] = str(lbl_data["Caminfo"]["Geometry"]["ShadowY"])

    field["ancillary"] = data
    json_data = json.dumps(field)
    return json_data


"""
cubデータから部分的にrefデータを抽出する関数。はなず...。

cubファイルを開き、指定ピクセル=座標（x、yは気を付ける）の反射率をref_csv（コンマ区切りの文字列）に格納。
ピクセル位置原点が画像の左上か左下か、わからん。
crismのcubデータも座標原点が左上か左下か、わからん。
"""
def base_json_getRef(params_json):
    ######## CRISM #########
    if params_json["obs_name"] == 'CRISM':

        # 新しいjson形式fieldを定義
        field = cl.OrderedDict()

        field["obs_ID"] = params_json["obs_ID"]
        field["obs_name"] = params_json["obs_name"]
        field["path"] = params_json["path"]
        field["Image_path"] = params_json["Image_path"]

        #cubファイルオープン！！
        #########
        cube_data = gdal.Open(field["path"]["main"]["cub"] , gdal.GA_ReadOnly) #cub ファイル名
        bandnumber_range = list(range(cube_data.RasterCount))
        NDV = cube_data.GetRasterBand(1).GetNoDataValue()
        ref_arr = []
        no_data_ref = 1
        first = 0

        if cube_data.GetRasterBand(30).ReadAsArray()[int(params_json["pixels"][1])][int(params_json["pixels"][0])] == NDV:
            no_data_ref = 1
        else:
            no_data_ref = 0
            for bandnumber_i in bandnumber_range:
                if first == 0:
                    first = 1
                    continue
                ref = cube_data.GetRasterBand(bandnumber_i+1).ReadAsArray()[int(params_json["pixels"][1])][int(params_json["pixels"][0])]
                if ref == NDV:
                    ref_arr.append(-1)
                else:
                    ref_arr.append(ref)
        
        #　反射率の格納
        if no_data_ref != 1:
            ref_csv=",".join(list(map(str,ref_arr)))
        else:
            ref_csv = -1
        #########
        
        field["pixels"] = params_json["pixels"]
        field["Image_size"] = [cube_data.RasterXSize,cube_data.RasterYSize]

        # 反射率
        field["reflectance"] = ref_csv

        bandnumber_range.pop(0)
        field["band_number"] = bandnumber_range
        field["band_bin_center"] = params_json["wavelength"]

        #cubファイルオープン！！
        cube_data2 = gdal.Open(field["path"]["derived"]["cub"], gdal.GA_ReadOnly)

        x = cube_data2.GetRasterBand(5).ReadAsArray()[int(params_json["pixels"][1])][int(params_json["pixels"][0])]
        y = cube_data2.GetRasterBand(4).ReadAsArray()[int(params_json["pixels"][1])][int(params_json["pixels"][0])]
        field["coordinate"] = [float(x),float(y)]

        json_data = json.dumps(field)
        return json_data